In [24]:
from selenium import webdriver
import base64
import re
from PIL import Image
import pandas as pd
from bs4 import BeautifulSoup as bs
import cv2
from selenium.webdriver.firefox.options import Options
from geradorPis import geradorDePisPasep
from random import randint
from quebrarCaptcha import quebrarCaptcha
import pandas as pd
import time
import sqlalchemy as sa
import psycopg2


In [25]:
option = Options()
option.headless = False
navegador = webdriver.Firefox(options=option,keep_alive=False)
navegador.delete_all_cookies()
navegador.get('http://rais.gov.br/sitio/consulta_trabalhador_identificacao.jsf')




# navegador.find_element_by_xpath('//*[@id="j_idt42:pis"]').click()




In [26]:
time.sleep(3)

In [27]:
src = navegador.page_source

In [28]:
padrao = '.id="j_idt[0-9]+:pis"'

In [29]:
padrao2 = '.name="j_idt[0-9]+:j_idt[0-9]+".value="Avançar"'

In [30]:
resultado = re.search(padrao, src)
resultado = resultado.group().split('="')[1].split('"')[0]
resultado2 = re.search(padrao2,src)
resultado2 = resultado2.group().split('="')[1].split('"')[0]

In [31]:
resultado2

'j_idt40:j_idt48'

In [32]:
navegador.find_element_by_name(resultado).send_keys(f'20195997101')

/tmp/ipykernel_700466/507650300.py:1: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  navegador.find_element_by_name(resultado).send_keys(f'20195997101')


In [33]:
navegador.find_element_by_id('btnRecarregar_captcha_serpro_gov_br').click()

/tmp/ipykernel_700466/3228145220.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  navegador.find_element_by_id('btnRecarregar_captcha_serpro_gov_br').click()


In [34]:
time.sleep(2)
img = navegador.find_element_by_xpath('//*[@id="img_captcha_serpro_gov_br"]').get_attribute('src').split(',')[1]
img=base64.b64encode(base64.b64decode(img))
fh = open(f"resolver/imagem.png", "wb")
fh.write(base64.decodebytes(img))
fh.close()
imagem = cv2.imread("resolver/imagem.png",1)
img_scale_up = cv2.resize(imagem, (0, 0), fx=3, fy=3)
cv2.imwrite(f'resolver/imagem.png',img_scale_up)

/tmp/ipykernel_700466/22805359.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  img = navegador.find_element_by_xpath('//*[@id="img_captcha_serpro_gov_br"]').get_attribute('src').split(',')[1]


True

In [35]:

captcha = quebrarCaptcha()


R18iar


In [36]:
navegador.find_element_by_xpath('//*[@id="txtTexto_captcha_serpro_gov_br"]').send_keys(f'{captcha}')
navegador.find_element_by_name(resultado2).click()

/tmp/ipykernel_700466/3595506688.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  navegador.find_element_by_xpath('//*[@id="txtTexto_captcha_serpro_gov_br"]').send_keys(f'{captcha}')
/tmp/ipykernel_700466/3595506688.py:2: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  navegador.find_element_by_name(resultado2).click()


In [37]:
trabalhador = navegador.page_source

In [38]:
try:
    validador = navegador.find_element_by_xpath('/html/body/div[2]/div[5]/ul/li').get_attribute('innerHTML')
except:
    validador = None

/tmp/ipykernel_700466/1952604686.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  validador = navegador.find_element_by_xpath('/html/body/div[2]/div[5]/ul/li').get_attribute('innerHTML')


In [39]:
# validador = validador.strip()
validador

AttributeError: 'NoneType' object has no attribute 'strip'

In [46]:
nome = navegador.find_element_by_xpath('//*[@id="j_idt27"]/div[2]/div/p').get_attribute('innerHTML').split('-')[1].strip()

/tmp/ipykernel_700466/1248569906.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  nome = navegador.find_element_by_xpath('//*[@id="j_idt27"]/div[2]/div/p').get_attribute('innerHTML').split('-')[1].strip()


In [47]:
nome

'CHRISTHIAN MICHELL BERNARDO SOUZA'